In [1]:
pip install pandas geopy folium

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from folium.plugins import MarkerCluster

# Step 1: Event data
data = [
    {"Date": "2021-11-07", "Location": "Istanbul, Turkey", "Distance": 15, "Event Type": "Running"},
    {"Date": "2022-03-27", "Location": "Istanbul, Turkey", "Distance": 10, "Event Type": "Running"},
    {"Date": "2025-10-02", "Location": "Istanbul, Turkey", "Distance": 5, "Event Type": "Swimming"},
    {"Date": "2022-11-06", "Location": "Istanbul, Turkey", "Distance": 15, "Event Type": "Running"},
    {"Date": "2023-05-14", "Location": "Berlin, Germany", "Distance": 10, "Event Type": "Running"},
    {"Date": "2023-08-27", "Location": "Berlin, Germany", "Distance": 3.5, "Event Type": "Swimming"},
    {"Date": "2024-04-25", "Location": "Berlin, Germany", "Distance": 5, "Event Type": "Running"},
    {"Date": "2024-04-28", "Location": "Potsdam, Germany", "Distance": 10, "Event Type": "Running"},
    {"Date": "2024-05-05", "Location": "Berlin, Germany", "Distance": 10, "Event Type": "Running"},
    {"Date": "2024-07-14", "Location": "Berlin, Germany", "Distance": 3.5, "Event Type": "Swimming"},
    {"Date": "2024-07-28", "Location": "Lindau, Germany", "Distance": 2.8, "Event Type": "Swimming"},
    {"Date": "2024-09-08", "Location": "Berlin, Germany", "Distance": 10, "Event Type": "Running"},
    {"Date": "2025-03-09", "Location": "Lisbon, Portugal", "Distance": 10, "Event Type": "Running"},
    {"Date": "2025-05-11", "Location": "Berlin, Germany", "Distance": 10, "Event Type": "Running"},
    {"Date": "2025-06-14", "Location": "Nienburg, Germany", "Distance": 10, "Event Type": "Running"},
    {"Date": "2025-07-13", "Location": "Berlin, Germany", "Distance": 3.5, "Event Type": "Swimming"}
    {"Date": "2025-09-29", "Location": "Copenhagen, Denmark", "Distance": 2, "Event Type": "Swimming"}
]

df = pd.DataFrame(data)

# Step 2: Geocode
geolocator = Nominatim(user_agent="sports_event_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df["Coordinates"] = df["Location"].apply(geocode)
df["Latitude"] = df["Coordinates"].apply(lambda loc: loc.latitude if loc else None)
df["Longitude"] = df["Coordinates"].apply(lambda loc: loc.longitude if loc else None)

# Step 3: Create map
m = folium.Map(location=[50, 10], zoom_start=4)

# Inject Font Awesome 5 via HTML hack (for fa-swimmer and fa-running)
fontawesome_link = '<link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.15.4/css/all.css">'
m.get_root().html.add_child(folium.Element(fontawesome_link))

marker_cluster = MarkerCluster().add_to(m)

# Step 4: Add icons with custom class
for _, row in df.iterrows():
    if pd.notnull(row["Latitude"]) and pd.notnull(row["Longitude"]):
        popup = f"{row['Date']}<br>{row['Location']}<br>{row['Distance']} km<br>{row['Event Type']}"

        if row["Event Type"].lower() == "swimming":
            icon = "fa-swimmer"
            color = "blue"
        else:
            icon = "fa-running"
            color = "black"

        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=popup,
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(marker_cluster)

# Step 5: Save map
m.save("sports_events_map.html")
print("✅ Map saved with swimmer and runner icons.")

✅ Map saved with swimmer and runner icons.
